In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the heart dataset
data = pd.read_csv('heart.csv')
X = data.drop(columns=['target']).values
y = data['target'].values

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

# Create DataLoader for batching
def get_dataloader(batch_size):
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

# Define MLP model
class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_layers, activation):
        super(MLPModel, self).__init__()
        layers = []
        for i in range(len(hidden_layers)):
            layers.append(nn.Linear(input_dim if i == 0 else hidden_layers[i-1], hidden_layers[i]))
            if activation == "linear":
                pass  # No activation
            elif activation == "sigmoid":
                layers.append(nn.Sigmoid())
            elif activation == "relu":
                layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], 2))  # Output layer (binary classification)
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Training function
def train_model(model, train_loader, test_loader, epochs, lr):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")

    # Evaluate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Experiment configurations
hidden_layer_options = [[4], [8], [16], [4, 4], [8, 8], [16, 16], [4, 4, 4], [8, 8, 8], [16, 16, 16]]
activation_options = ["linear", "sigmoid", "relu"]
epoch_options = [1, 10, 25]
lr_options = [10, 1, 0.1]
batch_size_options = [16, 32, 64]

# Run experiments
results = []
for hidden_layers in hidden_layer_options:
    for activation in activation_options:
        for epochs in epoch_options:
            for lr in lr_options:
                for batch_size in batch_size_options:
                    print(f"\nTesting configuration: Hidden Layers={hidden_layers}, Activation={activation}, Epochs={epochs}, LR={lr}, Batch Size={batch_size}")
                    train_loader, test_loader = get_dataloader(batch_size)
                    model = MLPModel(input_dim=X_train.shape[1], hidden_layers=hidden_layers, activation=activation).to(device)
                    accuracy = train_model(model, train_loader, test_loader, epochs, lr)
                    results.append((hidden_layers, activation, epochs, lr, batch_size, accuracy))

# Print all results
print("\nFinal Results:")
for result in results:
    print(f"Hidden Layers={result[0]}, Activation={result[1]}, Epochs={result[2]}, LR={result[3]}, Batch Size={result[4]}, Accuracy={result[5]:.2f}%")
